In [1]:
import warnings

warnings.filterwarnings("ignore") # specify to ignore warning messages

import pandas as pd
import matplotlib.pyplot as plt
import itertools as it
import datetime
import math

import statsmodels.api as sm

In [2]:
xl = pd.ExcelFile('input_data.xlsx')

data = {sheet_name: xl.parse(sheet_name) for sheet_name in xl.sheet_names}

days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

res = []

for name in xl.sheet_names:
    mylist = map(list, zip(*data[name].values))
    aux = list(it.chain(*mylist))
    del aux[:24]
    res.append(aux)

res = list(it.chain(*res))

days = [days for i in range(0, 4)]
days_repeated = days * 4

In [3]:
# Tune Seasonal ARIMA model
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0,2)

# Generate all different combinations of p, q and q triplets
pdq = list(it.product(p, d, q))
print(pdq)

# Generate all different combinations of seasonal p, q and q triplets
# Seasonality is one week (24*7 = 168 hours)
seasonal_pdq = [(x[0], x[1], x[2], 168) for x in list(it.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
Examples of parameter combinations for Seasonal ARIMA...
SARIMAX: (0, 0, 1) x (0, 0, 1, 168)
SARIMAX: (0, 0, 1) x (0, 1, 0, 168)
SARIMAX: (0, 1, 0) x (0, 1, 1, 168)
SARIMAX: (0, 1, 0) x (1, 0, 0, 168)


In [4]:
result_list = []

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(res,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = mod.fit()

            print('ARIMA{}x{}168 - AIC:{}'.format(param, param_seasonal, round(results.aic,2)))
            result_list.extend([param, param_seasonal, round(results.aic,2)])
        except:
            print('error')
            continue
            
print('Done!')

In [30]:
print_result = zip(*[iter(result_list)]*3) 
print_result.sort(key=lambda x: x[2])

print('Result summary:\n')
print('((p, d, q), (P, D, Q, S), AIC)')
print('------------------------------')
for item in print_result:
    print item

Result summary:

((p, d, q), (P, D, Q, S), AIC)
------------------------------
((1, 1, 1), (1, 1, 0, 168), 3818.95)
((1, 0, 0), (1, 1, 0, 168), 3835.74)
((1, 0, 1), (1, 1, 0, 168), 3836.72)
((0, 0, 1), (1, 1, 0, 168), 3854.2)
((0, 1, 1), (1, 1, 0, 168), 3877.28)
((1, 1, 0), (1, 1, 0, 168), 3912.87)
((0, 0, 0), (1, 1, 0, 168), 3915.73)
((0, 1, 0), (1, 1, 0, 168), 3939.41)
((1, 1, 1), (0, 1, 0, 168), 5934.29)
((1, 0, 1), (0, 1, 0, 168), 5946.06)
((1, 1, 1), (1, 0, 0, 168), 5948.39)
((0, 0, 1), (0, 1, 0, 168), 5953.08)
((1, 0, 0), (0, 1, 0, 168), 5956.36)
((1, 0, 0), (1, 0, 0, 168), 5956.75)
((1, 0, 1), (1, 0, 0, 168), 5957.24)
((0, 0, 1), (1, 0, 0, 168), 5973.03)
((0, 1, 1), (0, 1, 0, 168), 6007.22)
((0, 1, 1), (1, 0, 0, 168), 6028.44)
((0, 0, 0), (1, 0, 0, 168), 6050.84)
((0, 0, 0), (0, 1, 0, 168), 6053.09)
((1, 1, 0), (0, 1, 0, 168), 6084.03)
((1, 1, 0), (1, 0, 0, 168), 6085.09)
((0, 1, 0), (0, 1, 0, 168), 6114.69)
((0, 1, 0), (1, 0, 0, 168), 6121.6)
((1, 0, 1), (0, 0, 0, 168), 8866.63